In [5]:
import cc3d
import netCDF4
import glob
import numpy
import sys
import datetime
import pickle

# Loading the data

In [6]:
def load_imerg(file_path, variable_name):
    netcdf = netCDF4.Dataset(file_path, 'r', format='NETCDF4')
    data = netcdf.groups['Grid'][variable_name][:][0].T[::-1]
    file_header = netcdf.FileHeader.split(';\n')
    file_header.remove('')
    header = {r.split('=')[0]: r.split('=')[1] for r in file_header}
    return data, header

In [7]:
file_paths = sorted(glob.glob('/tablespace/xcal/imerg/3B-HHR.MS.MRG.3IMERG*'))
file_paths = sorted(glob.glob('data/imerg/3B-HHR.MS.MRG.3IMERG*'))
variable_name = 'precipitationCal'

stack = []
headers = []
timestamps = []
for file_path in file_paths:
    print('{}'.format(len(file_paths)-len(stack)), end='\r')
    sys.stdout.flush()
    data, header = load_imerg(file_path, variable_name)
    stack.append(data)
    headers.append(header)
    timestamps.append(header['StartGranuleDateTime'])
    
data = numpy.ma.array(stack)

timestamps_dt = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.000Z') for date in timestamps]
timestamps_np = numpy.array(timestamps_dt, dtype='datetime64[m]')

# Thresholding and CCLs

In [8]:
thresh = 1
data[data<thresh] = 0
data[data>=thresh] = 1

connectivity = 6 
min_voxels = 100

In [9]:
labels = cc3d.connected_components(data, 
                                   delta=0,
                                   connectivity=connectivity)

In [10]:
labels = cc3d.dust(labels, 
                   threshold=min_voxels, 
                   connectivity=connectivity, 
                   in_place=False)

In [11]:
labels = numpy.ma.array(labels, mask=data.mask)
labels = numpy.ma.masked_where(labels==0, labels)

In [ ]:
largest, N = cc3d.largest_k(labels, 
                            k=10, 
                            connectivity=connectivity, 
                            delta=0, 
                            return_N=True)

largest = numpy.ma.masked_where(largest==0, largest)

# Pickle

In [27]:
with open('pickles/largest.pickle', 'wb') as f:
    pickle.dump(largest, f)

with open('pickles/labels.pickle', 'wb') as f:
    pickle.dump(labels, f)
    
with open('pickles/timestamps.pickle', 'wb') as f:
    pickle.dump(timestamps_dt, f)